Database creation

In [1]:
import os

print(os.getcwd())

/Users/tom/Documents/Coding/AiCore/DataCollectionPipeline/setup


In [4]:


conda install psycopg2

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [8]:
from sqlalchemy import create_engine


DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'aicoredb.cz91qpjes5tm.us-east-1.rds.amazonaws.com' # Change it for your AWS endpoint
USER = 'postgres'
PASSWORD = ""
PORT = 5432
DATABASE = 'lms_db'

print(PASSWORD)

engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")



TootingCoffeePatio9(


In [4]:
engine.connect()

In [5]:
import pandas as pd
import json

# items have to be in a  list to get an auto assinged index value

with open('../scraped_data/Luke Mussert/data.json') as json_file:
    data = json.load(json_file)
    print(data)

with open('../scraped_data/Tom McHale/data.json') as json_file:
    data = json.load(json_file)
    print(data)

players = pd.DataFrame(data, columns=['PlayerName', 'UUID', 'PlayerLink'], index=[0])
players = players.rename(columns={"PlayerName": "name", "UUID": "uuid", "PlayerLink" : "lms_profile_link" })

scorecard = pd.DataFrame(data, columns=['ScorecardIds', 'UUID'])
scorecard = scorecard.rename(columns={"ScorecardIds": "scorecard_id", "UUID": "uuid"})


players['name'] = players['name'].astype('string')
players['uuid'] = players['uuid'].astype('string')
players['lms_profile_link'] = players['lms_profile_link'].astype('string')

scorecard['scorecard_id'] = scorecard['scorecard_id'].astype('int64')
scorecard['uuid'] = scorecard['uuid'].astype('string')

if data['ScorecardBowlingData']:
    bowling_data = pd.DataFrame(data['ScorecardBowlingData'])
    bowling_data = bowling_data.rename(columns={"Overs": "overs", "Runs": "runs_conceeded", "Wickets": "wickets", "Maidens": "maidens", "Economy": "economy"})
    combine_bowling = pd.merge(scorecard, bowling_data, left_index=True, right_index=True)
    combine_bowling['overs'] = combine_bowling['overs'].astype('float64')
    combine_bowling['runs_conceeded'] = combine_bowling['runs_conceeded'].astype('int64')
    combine_bowling['wickets'] = combine_bowling['wickets'].astype('int64')
    combine_bowling['maidens'] = combine_bowling['maidens'].astype('int64')
    combine_bowling['economy'] = combine_bowling['economy'].astype('float64')


if data['ScorecardBattingData']:
    batting_data = pd.DataFrame(data['ScorecardBattingData'])
    batting_data = batting_data.rename(columns={"How Out": "how_out", "Runs": "runs_scored", "Balls": "balls_faced", "Fours": "fours", "Sixes": "sixes", "SR": "strike_rate"})
    combine_batting = pd.merge(scorecard, batting_data, left_index=True, right_index=True)
    combine_batting['how_out'] = combine_batting['how_out'].astype('category')
    combine_batting['runs_scored'] = combine_batting['runs_scored'].astype('int64')
    combine_batting['balls_faced'] = combine_batting['balls_faced'].astype('int64')
    combine_batting['fours'] = combine_batting['fours'].astype('int64')
    combine_batting['sixes'] = combine_batting['sixes'].astype('int64')
    combine_batting['strike_rate'] = combine_batting['strike_rate'].astype('float64')

uuid = pd.DataFrame(data, columns=['UUID'], index=[0])
awards = pd.DataFrame(data['Awards'], index=[0])
uuid = uuid.rename(columns={"UUID": "uuid"})
awards = awards.rename(columns={"MostValuablePlayer": "most_valuable_player", "MostValuableBatter": "most_valuable_batter","MostValuableBowler": "most_valuable_bowler"})
combine_awards = pd.merge(uuid, awards, left_index=True, right_index=True)
combine_awards['uuid'] = combine_awards['uuid'].astype('string')



combine_batting.info()

{'PlayerName': 'Luke Mussert', 'UUID': '4ce04769-f2bf-42ba-8f56-b5debd82daaf', 'PlayerLink': 'https://www.lastmanstands.com/cricket-player/t20?playerid=265273', 'ImageLink': ['https://www.lastmanstands.com/images/player-silhouette.png'], 'ScorecardIds': ['346950', '346946', '346941', '346937', '346936', '327279', '327273', '327286', '327268', '308956', '308958', '308952', '308945', '308942', '308939', '308934'], 'ScorecardBattingData': [{'How Out': 'Not Out', 'Runs': '1', 'Balls': '2', 'Fours': '0', 'Sixes': '0', 'SR': '50.00'}, {'How Out': 'Not Out', 'Runs': '33', 'Balls': '28', 'Fours': '2', 'Sixes': '1', 'SR': '117.86'}, {'How Out': 'Run Out', 'Runs': '20', 'Balls': '13', 'Fours': '2', 'Sixes': '0', 'SR': '153.85'}, {'How Out': 'Bowled', 'Runs': '37', 'Balls': '23', 'Fours': '3', 'Sixes': '1', 'SR': '160.87'}, {'How Out': 'Not Out', 'Runs': '4', 'Balls': '1', 'Fours': '1', 'Sixes': '0', 'SR': '400.00'}, {'How Out': 'Caught', 'Runs': '20', 'Balls': '19', 'Fours': '2', 'Sixes': '0', '

In [59]:
combine_bowling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   scorecard_id    2 non-null      int64  
 1   uuid            2 non-null      string 
 2   overs           2 non-null      float64
 3   runs_conceeded  2 non-null      int64  
 4   wickets         2 non-null      int64  
 5   maidens         2 non-null      int64  
 6   economy         2 non-null      float64
dtypes: float64(2), int64(4), string(1)
memory usage: 240.0 bytes


Upload to database

In [7]:
combine_batting['How Out'] = combine_batting['How Out'].astype('category')
combine_batting['ScorecardIds'] = combine_batting['ScorecardIds'].astype('int64')
combine_batting['UUID'] = combine_batting['UUID'].astype('string')
combine_batting['Runs'] = combine_batting['Runs'].astype('int64')
combine_batting['Balls'] = combine_batting['Balls'].astype('int64')
combine_batting['Fours'] = combine_batting['Fours'].astype('int64')
combine_batting['Sixes'] = combine_batting['Sixes'].astype('int64')
combine_batting['SR'] = combine_batting['SR'].astype('float64')

In [13]:
combine_bowling['ScorecardIds'] = combine_bowling['ScorecardIds'].astype('int64')
combine_bowling['UUID'] = combine_bowling['UUID'].astype('string')
combine_bowling['Overs'] = combine_bowling['Overs'].astype('float64')
combine_bowling['Runs'] = combine_bowling['Runs'].astype('int64')
combine_bowling['Wickets'] = combine_bowling['Wickets'].astype('int64')
combine_bowling['Maidens'] = combine_bowling['Maidens'].astype('int64')
combine_bowling['Economy'] = combine_bowling['Economy'].astype('float64')

In [22]:
players['PlayerName'] = players['PlayerName'].astype('string')
players['UUID'] = players['UUID'].astype('string')
players['PLayerLink'] = players['PlayerLink'].astype('string')


In [30]:
combine_awards['UUID'] = combine_awards['UUID'].astype('string')


In [31]:
combine_awards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   UUID                1 non-null      string
 1   MostValuablePlayer  1 non-null      int64 
 2   MostValuableBatter  1 non-null      int64 
 3   MostValuableBowler  1 non-null      int64 
dtypes: int64(3), string(1)
memory usage: 148.0 bytes


In [9]:
players.to_sql('players', engine, if_exists='append')
if combine_batting is not None:
    combine_batting.to_sql('batting', engine, if_exists='append')
if combine_bowling is not None:
    combine_bowling.to_sql('bowling', engine, if_exists='append')
combine_awards.to_sql('awards', engine, if_exists='append')

1

In [3]:
user_choice = input(' \n Please press 1 for local storage, 2 for online (RDS) or 3 for both \n')
if user_choice == "1" or user_choice == "3":
    print("locals")

locals
